### MNIST Loader
This is python 3 vesrion of the code . The orignal code was in python 2.7 in link below
https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/src/mnist_loader.py

In [6]:
"""
mnist_loader
~~~~~~~~~~~~
A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""
import random
import numpy as np
#### Libraries
# Standard library
import pickle
import gzip



def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.
    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.
    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.
    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('mnist.pkl.gz', 'rb')
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    
    training_data, validation_data, test_data = u.load()
    
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.
    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.
    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [ ]:
train, valid, test = load_data_wrapper()
X_train, y_label = zip(*train)
print('Number of Train samples {}'.format(len(X_train)))
print('Each Train sample has shape {}'.format(X_train[0].shape))
print('Label Shape', y_label[0].shape)

In [ ]:

print('Sample Train\n',X_train[7])

In [ ]:
#print('Sample Label\n',y_label[0])

### Model Creation

In [2]:


class Network(object):
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes  #Each element in list represent number of nuerons is that layer
        self.biases  = [np.random.randn(y, 1) for y in sizes[1:] ] #For first layer bias not set. 
        #Create list of weight arrays. Each element in list is array of dimension (y,x) where y is number of
        # nuerons in next layer and  x is number of nuerons in previous layer
        self.weights = [np.random.randn(y,x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

    # each x is numpy array of dimension of (number of input layer, 1)         
        for x, y in mini_batch:
             delta_nabla_b, delta_nabla_w = self.backprop(x, y)
             nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]           
             nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
   
        self.weights = [ w- (eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases =  [b- (eta/len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
    
       
    def SGD(self, train, epochs, mini_batch_size, eta, test = None):
        if test:
            n_test = len(test)

        n = len(train)    
        for j in range(epochs):
            random.shuffle(train)
        #   Suppose number of training samples is 50, and mini batch size is 10. 
        #   This will create list of  5(50/10) batches with each batch containing 10 samples
            mini_batches = [ train[k: k+mini_batch_size]  for k in range(0, n, mini_batch_size) ]

            for mini_batch in mini_batches:                
                self.update_mini_batch(mini_batch, eta)          
          
            if test:
               print('Epoch {}: {}/ {}'.format(j, self.evaluate(test), n_test))
            else:
               print( 'Epoch {} complete'.format(j))

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]         
        activation = x

        #Store activations layer by layer
        activations = [x]
        zs = [] #Store z vectors which is dot product of wieght and activatons

    #  each loop pass corresponds to bias and wieghts for each layer. 
    #  the first hidden layer is processes first followed by next and so on
        for b, w in zip(self.biases, self.weights):                
             z= np.dot(w, activation) + b
             zs.append(z)
             activation = sigmoid(z)
             activations.append(activation)



    #   start of back propogation:Set the values for last layer
        delta = self.cost_derivative(activations[-1], y)  * sigmoid_prime(zs[-1]) 
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

    #    Do backpropogation for rest of layers
        for l in range(2, self.num_layers):
             z = zs[-l]
             sp = sigmoid_prime(z)
             delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
             nabla_b[-l] = delta
             nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return nabla_b, nabla_w

    def evaluate(self, test):
        #Return number of test inputs for which predictions are correct
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test]
        return sum(int(x == y) for (x, y) in test_results)
    def feedforward(self, a):
        for b ,w in zip(self.biases, self.weights):            
            a = sigmoid(np.dot(w, a) + b)
        return a
  
    def cost_derivative(self, output_activations, y):
        return(output_activations - y)    
    
def sigmoid(z):
     return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    # derivative of sigmoid function
      return sigmoid(z) * (1- sigmoid(z))     

## Training

In [7]:
train, valid, test = load_data_wrapper() 
# Craete a Network with 784 input layers, 30 Hidden layers and 10 output layers
net = Network([784, 30, 10])


net.SGD(train,
        epochs = 30,
        mini_batch_size = 10,
        eta =3,
        test = test,        
        )


Epoch 0: 9046/ 10000
Epoch 1: 9218/ 10000
Epoch 2: 9274/ 10000
Epoch 3: 9324/ 10000
Epoch 4: 9373/ 10000
Epoch 5: 9370/ 10000
Epoch 6: 9416/ 10000
Epoch 7: 9424/ 10000
Epoch 8: 9436/ 10000
Epoch 9: 9456/ 10000
Epoch 10: 9441/ 10000
Epoch 11: 9458/ 10000
Epoch 12: 9468/ 10000
Epoch 13: 9461/ 10000
Epoch 14: 9467/ 10000
Epoch 15: 9451/ 10000
Epoch 16: 9492/ 10000
Epoch 17: 9505/ 10000
Epoch 18: 9487/ 10000
Epoch 19: 9497/ 10000
Epoch 20: 9488/ 10000
Epoch 21: 9505/ 10000
Epoch 22: 9489/ 10000
Epoch 23: 9493/ 10000
Epoch 24: 9477/ 10000
Epoch 25: 9482/ 10000
Epoch 26: 9495/ 10000
Epoch 27: 9502/ 10000
Epoch 28: 9505/ 10000
Epoch 29: 9499/ 10000
